## Esercizi
1. Caricare ed ispezionare il dataset Iris
2. Effettuare preprocessing, split e normalizzazione del dataset
3. Creare dataset e dataloader
4. Creare una rete neurale per risolvere il task di classificazione
5. Allenare la rete
6. Testare la rete
7. Calcolare l'accuracy ottenuta dalla rete

### Inizializzazione

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import numpy as np
import random
%matplotlib inline

In [32]:
seed = 156
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)


1. Caricare ed ispezionare il dataset Iris

Il Dataset Iris è un famoso dataset di classificazione.

Contiene le misure di stelo e petalo di diversi fiori iris e la loro specie.

Lo scopo è quello di allenare una rete che, prendendo i dati dei fiori, sia in grado di predirre la loro specie

In [33]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df.columns = iris.feature_names
df['Species'] = iris.target

In [34]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [35]:
df.Species.value_counts()

Species
0    50
1    50
2    50
Name: count, dtype: int64

In [36]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [37]:
df.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Species              0
dtype: int64

2. Effettuare preprocessing, split e normalizzazione del dataset

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[:-1]],
                                                    pd.DataFrame(df['Species'], columns=['Species']),
                                                    test_size = 0.3, random_state = seed)



In [39]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(105, 4)
(45, 4)
(105, 1)
(45, 1)


In [40]:
X_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
21,5.1,3.7,1.5,0.4
116,6.5,3.0,5.5,1.8
115,6.4,3.2,5.3,2.3
136,6.3,3.4,5.6,2.4
68,6.2,2.2,4.5,1.5


In [41]:
Y_train.head()

,Species
21,0
116,2
115,2
136,2
68,1


In [42]:
# Reset degli indici
X_train = X_train.reset_index().drop(columns=['index'])
X_test = X_test.reset_index().drop(columns=['index'])

Y_train = Y_train.reset_index().drop(columns=['index'])
Y_test= Y_test.reset_index().drop(columns=['index'])



In [43]:
max_df = X_train.max()
min_df = X_train.min()

X_train_norm = (X_train - min_df)/(max_df - min_df)

X_train_norm.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,105.000000,105.000000,105.000000,105.000000
mean,0.421164,0.462302,0.461905,0.449206
std,0.221155,0.178424,0.320129,0.325734
min,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.375000,0.071429,0.083333
50%,0.416667,0.416667,0.589286,0.500000
75%,0.583333,0.583333,0.714286,0.708333
max,1.000000,1.000000,1.000000,1.000000


In [44]:
# Anche il test set viene normalizzato allo stesso modo
X_test_norm = (X_test - min_df)/(max_df - min_df)

3. Creare dataset e dataloader

In [45]:
train_dataset = data_utils.TensorDataset(torch.tensor(X_train_norm.values, dtype = torch.float32),
                                         torch.tensor(Y_train.values, dtype = torch.long))
trainloader = data_utils.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = data_utils.TensorDataset(torch.tensor(X_test_norm.values, dtype = torch.float32),
                                        torch.tensor(Y_test.values, dtype = torch.long))
testloader = data_utils.DataLoader(test_dataset, batch_size=32, shuffle=False)

4. Creare una rete neurale per risolvere il task di classificazione

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer_1 = nn.Linear(in_features=4, out_features=64)    #(4 + 1)*64 = 320
        self.layer_2 = nn.Linear(in_features=64, out_features=32)   #(64 + 1)*32 = 2080

        self.layer_3 = nn.Linear(in_features=32, out_features=3)    #(32 + 1)*3 = 99

    def forward(self, x):
        x = self.layer_1(x) #[32, 4] -> [32, 64]
        x = F.relu(x) #[32, 64] -> [32, 64]
        x = self.layer_2(x) #[32, 64] -> [32, 32]
        x = F.relu(x) #[32, 32] -> [32, 32]
        x = self.layer_3(x) #[32, 32] -> [32, 3]
        return x

In [46]:
net = MLP()

5. Allenare la rete

In [47]:
# Loss function
loss_fn = nn.CrossEntropyLoss() # siamo in un problema di classificazione

# SGD optimizer
optimizer = torch.optim.Adam(params=net.parameters(),
                            lr=0.001)

In [ ]:
net.train()

In [49]:
epochs = 50

for epoch in range(epochs):

    loss_epoch = 0
    for i, data in enumerate(trainloader, 0):

      X = data[0] #[32, 4]
      y = data[1] #[32 ,3]


      # 1. Forward pass
      y_pred = net(X) #[32, 3]
      # 2. Calculo della loss
      loss = loss_fn(y_pred, y.squeeze())

      loss_epoch += loss

      # 3. Azzeramento dei gradienti
      optimizer.zero_grad()

      # 4. Backpropagation
      loss.backward()

      # 5. Ottimizzazione
      optimizer.step()

    print(f"Epoca: {epoch} |  Train Loss: {loss_epoch/len(trainloader)}")

Epoca: 0 |  Train Loss: 1.0910943746566772
Epoca: 1 |  Train Loss: 1.0787756443023682
Epoca: 2 |  Train Loss: 1.0639933347702026
Epoca: 3 |  Train Loss: 1.0515247583389282
Epoca: 4 |  Train Loss: 1.0425548553466797
Epoca: 5 |  Train Loss: 1.0272349119186401
Epoca: 6 |  Train Loss: 1.0125243663787842
Epoca: 7 |  Train Loss: 0.993003249168396
Epoca: 8 |  Train Loss: 0.97456294298172
Epoca: 9 |  Train Loss: 0.9642822742462158
Epoca: 10 |  Train Loss: 0.943211555480957
Epoca: 11 |  Train Loss: 0.9131188988685608
Epoca: 12 |  Train Loss: 0.8883334398269653
Epoca: 13 |  Train Loss: 0.8582672476768494
Epoca: 14 |  Train Loss: 0.8424766063690186
Epoca: 15 |  Train Loss: 0.8057124614715576
Epoca: 16 |  Train Loss: 0.7560645937919617
Epoca: 17 |  Train Loss: 0.7303094863891602
Epoca: 18 |  Train Loss: 0.743316650390625
Epoca: 19 |  Train Loss: 0.6599864959716797
Epoca: 20 |  Train Loss: 0.6638801693916321
Epoca: 21 |  Train Loss: 0.5983784794807434
Epoca: 22 |  Train Loss: 0.6065179705619812
Epo

6. Testare la rete


In [50]:
#Imposiamo la rete in modalità eval
net.eval()

MLP(
  (layer_1): Linear(in_features=4, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=32, bias=True)
  (layer_3): Linear(in_features=32, out_features=3, bias=True)
)

In [51]:
loss_test = 0
for i, data in enumerate(testloader, 0):

  X = data[0] #[32, 4]
  y = data[1] #[32, 3]

  with torch.no_grad():

    # 1. Forward pass
    y_pred = net(X) #[32, 3]
    # 2. Calculo della loss
    loss = loss_fn(y_pred, y.squeeze())

  loss_test += loss
print(loss_test/len(testloader))

tensor(0.4124)


7. Calcolare l'accuracy ottenuta dalla rete

# Metriche di Classificazione
## Classificazione Binaria

![](https://miro.medium.com/max/1400/1*PPgItHcPSaskyjLMWFC-Kw.png
)

##$Accuracy$ = $\frac{TP+TN}{TP+FP+TN+FN}$

## Classificazione Categorica

![](https://www.researchgate.net/profile/Frank-Krueger-2/publication/314116591/figure/fig7/AS:614085901185031@1523420896093/Confusion-matrix-for-multi-class-classification-The-confusion-matrix-of-a_W640.jpg)

##$Accuracy$ = $\frac{diag(CM)}{sum(CM)}$

In [52]:
# conta le prediction corrette
correct_pred = 0
total_pred = 0

# non serve calcolare il gradiente
with torch.no_grad():
    for data in testloader:
        X = data[0]
        y = data[1]
        outputs = net(X)
        _, predictions = torch.max(outputs, 1)
        # collezioniamo le prediction
        for label, prediction in zip(y, predictions):
            if label == prediction:
                correct_pred += 1
            total_pred += 1


accuracy = 100 * correct_pred / total_pred
print(f'Accuracy : {accuracy:.1f} %')

Accuracy : 91.1 %
